
# Retail Analytics (pandas vs polars vs PySpark)

Este notebook usa o dataset **RetailStoreProductSalesDataset.csv** (15.000 linhas) para mostrar **equivalências práticas** entre:

- **pandas** (DataFrame em memória)
- **polars** (DataFrame/expressões colunar, bem rápido)
- **PySpark** (DataFrame distribuído)

Em cada seção você verá **a mesma análise feita 3 vezes** (uma por biblioteca), usando operações comuns do dia a dia.



## 0) Setup e leitura do dataset

> Ajuste o caminho se necessário. Aqui estou usando o arquivo montado no ambiente.


In [1]:

# Imports
import pandas as pd

# Polars
import polars as pl

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Caminho do dataset
CSV_PATH = r"RetailStoreProductSalesDataset.csv"

# Spark session
spark = (SparkSession.builder
         .appName("retail-analytics-pandas-polars-spark")
         .getOrCreate())

# Leitura: pandas
df_pd = pd.read_csv(CSV_PATH)

# Leitura: polars
df_pl = pl.read_csv(CSV_PATH)

# Leitura: spark
df_sp = (spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(CSV_PATH))

print("pandas shape:", df_pd.shape)
print("polars shape:", df_pl.shape)
print("spark count:", df_sp.count(), " | columns:", len(df_sp.columns))


pandas shape: (15000, 11)
polars shape: (15000, 11)
spark count: 15000  | columns: 11



## 1) Visão rápida e checagem de tipos

Só pra garantir que estamos olhando a mesma coisa nas 3 libs.


In [2]:

# pandas
display(df_pd.head(3))
print(df_pd.dtypes)

# polars
print(df_pl.head(3))
print(df_pl.schema)

# spark
df_sp.show(3, truncate=False)
df_sp.printSchema()


,Unnamed: 0,price,discount,promotion_intensity,footfall,ad_spend,competitor_price,stock_level,weather_index,customer_sentiment,return_rate
0,0,45.197454,5.514259,4.062653,277.017484,2559.073870,44.255411,1176.981397,7.442194,1.021762,0.067388
1,1,49.327512,6.572035,4.964657,250.760714,2536.417155,50.331704,1219.359497,6.759510,0.975142,0.082719
2,2,47.328457,6.972713,4.363191,263.130478,2552.952356,49.285996,1263.822738,5.941689,0.871567,0.061600


Unnamed: 0               int64
price                  float64
discount               float64
promotion_intensity    float64
footfall               float64
ad_spend               float64
competitor_price       float64
stock_level            float64
weather_index          float64
customer_sentiment     float64
return_rate            float64
dtype: object
shape: (3, 11)
┌─────┬───────────┬──────────┬─────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│     ┆ price     ┆ discount ┆ promotion_i ┆ … ┆ stock_leve ┆ weather_in ┆ customer_s ┆ return_rat │
│ --- ┆ ---       ┆ ---      ┆ ntensity    ┆   ┆ l          ┆ dex        ┆ entiment   ┆ e          │
│ i64 ┆ f64       ┆ f64      ┆ ---         ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│     ┆           ┆          ┆ f64         ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═════╪═══════════╪══════════╪═════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 0   ┆ 45.197454 ┆ 5.51


## 2) Somar uma coluna (ex.: total de `ad_spend`)

Pergunta: quanto foi investido em anúncios no período?


In [3]:

# pandas
total_ad_pd = df_pd["ad_spend"].sum()

# polars
total_ad_pl = df_pl["ad_spend"].sum()

# spark
total_ad_sp = df_sp.agg(F.sum("ad_spend").alias("total_ad_spend")).collect()[0]["total_ad_spend"]

print("Total ad_spend | pandas:", total_ad_pd)
print("Total ad_spend | polars:", total_ad_pl)
print("Total ad_spend | spark :", total_ad_sp)


Total ad_spend | pandas: 37574977.52731252
Total ad_spend | polars: 37574977.52731252
Total ad_spend | spark : 37574977.52731238



## 3) Filtrar linhas (ex.: dias com desconto alto e estoque baixo)

Filtro: `discount > 0.30` **e** `stock_level < 20`  
Isso pode indicar risco de ruptura + promoção agressiva.


In [4]:

# pandas
high_disc_low_stock_pd = df_pd[(df_pd["discount"] > 0.30) & (df_pd["stock_level"] < 20)]

# polars
high_disc_low_stock_pl = df_pl.filter((pl.col("discount") > 0.30) & (pl.col("stock_level") < 20))

# spark
high_disc_low_stock_sp = df_sp.filter((F.col("discount") > 0.30) & (F.col("stock_level") < 20))

print("Linhas filtradas | pandas:", len(high_disc_low_stock_pd))
print("Linhas filtradas | polars:", high_disc_low_stock_pl.height)
print("Linhas filtradas | spark :", high_disc_low_stock_sp.count())

display(high_disc_low_stock_pd.head(5))


Linhas filtradas | pandas: 0
Linhas filtradas | polars: 0
Linhas filtradas | spark : 0


,Unnamed: 0,price,discount,promotion_intensity,footfall,ad_spend,competitor_price,stock_level,weather_index,customer_sentiment,return_rate



## 4) Contar uma coluna (não nulos)

Vamos contar registros válidos em `customer_sentiment` (deve ser quase tudo, mas é um check rápido).


In [5]:

# pandas
count_sent_pd = df_pd["customer_sentiment"].count()

# polars
count_sent_pl = df_pl["customer_sentiment"].count()

# spark
count_sent_sp = df_sp.select(F.count("customer_sentiment").alias("cnt")).collect()[0]["cnt"]

print("Count customer_sentiment | pandas:", count_sent_pd)
print("Count customer_sentiment | polars:", count_sent_pl)
print("Count customer_sentiment | spark :", count_sent_sp)


Count customer_sentiment | pandas: 15000
Count customer_sentiment | polars: 15000
Count customer_sentiment | spark : 15000



## 5) Contar valores distintos (ex.: `weather_index` arredondado)

Como é numérico contínuo, contar distintos “cru” costuma ser gigante.  
Então vamos arredondar (`weather_index` com 1 casa) e contar distintos.


In [6]:

# pandas
weather_rounded_pd = df_pd["weather_index"].round(1)
nunique_weather_pd = weather_rounded_pd.nunique()

# polars
nunique_weather_pl = (df_pl
    .select(pl.col("weather_index").round(1).n_unique().alias("n_unique_weather_round1"))
    .item())

# spark
nunique_weather_sp = (df_sp
    .select(F.round("weather_index", 1).alias("weather_round1"))
    .agg(F.countDistinct("weather_round1").alias("n_unique_weather_round1"))
    .collect()[0]["n_unique_weather_round1"])

print("Distinct weather_index (round 1) | pandas:", nunique_weather_pd)
print("Distinct weather_index (round 1) | polars:", nunique_weather_pl)
print("Distinct weather_index (round 1) | spark :", nunique_weather_sp)


Distinct weather_index (round 1) | pandas: 116
Distinct weather_index (round 1) | polars: 116
Distinct weather_index (round 1) | spark : 116



## 6) Agrupar e somar (group by)

Vamos criar um *bucket* simples de sentimento e somar `footfall` por faixa.  
Isso dá uma ideia de “tráfego total” associado a diferentes níveis de sentimento.


In [7]:

# Função de bucketing (para manter a lógica simples e idêntica)
def sentiment_bucket(x):
    if x < -0.3:
        return "negativo"
    elif x > 0.3:
        return "positivo"
    return "neutro"

# pandas
df_pd_tmp = df_pd.copy()
df_pd_tmp["sent_bucket"] = df_pd_tmp["customer_sentiment"].apply(sentiment_bucket)
gb_pd = (df_pd_tmp.groupby("sent_bucket", as_index=False)["footfall"].sum()
         .sort_values("footfall", ascending=False))

# polars
df_pl_tmp = df_pl.with_columns(
    pl.when(pl.col("customer_sentiment") < -0.3).then(pl.lit("negativo"))
     .when(pl.col("customer_sentiment") > 0.3).then(pl.lit("positivo"))
     .otherwise(pl.lit("neutro"))
     .alias("sent_bucket")
)
gb_pl = (df_pl_tmp
         .group_by("sent_bucket")
         .agg(pl.col("footfall").sum().alias("footfall_sum"))
         .sort("footfall_sum", descending=True))

# spark
df_sp_tmp = (df_sp
    .withColumn(
        "sent_bucket",
        F.when(F.col("customer_sentiment") < -0.3, F.lit("negativo"))
         .when(F.col("customer_sentiment") > 0.3, F.lit("positivo"))
         .otherwise(F.lit("neutro"))
    )
)
gb_sp = (df_sp_tmp
         .groupBy("sent_bucket")
         .agg(F.sum("footfall").alias("footfall_sum"))
         .orderBy(F.col("footfall_sum").desc()))

display(gb_pd)
print(gb_pl)
gb_sp.show()


,sent_bucket,footfall
1,positivo,3.584063e+06
0,neutro,7.620255e+03


shape: (2, 2)
┌─────────────┬──────────────┐
│ sent_bucket ┆ footfall_sum │
│ ---         ┆ ---          │
│ str         ┆ f64          │
╞═════════════╪══════════════╡
│ positivo    ┆ 3.5841e6     │
│ neutro      ┆ 7620.254896  │
└─────────────┴──────────────┘
+-----------+------------------+
|sent_bucket|      footfall_sum|
+-----------+------------------+
|   positivo|3584062.5058610616|
|     neutro| 7620.254896064795|
+-----------+------------------+




## 7) Transformar colunas em linhas (melt / unpivot)

Exemplo: pegar um subconjunto de features e transformar para formato “long”  
para facilitar plots, comparações e checagem de distribuição.


In [8]:

FEATURES = ["price", "discount", "promotion_intensity", "footfall", "ad_spend"]

# Para não explodir tamanho, vamos usar uma amostra
sample_pd = df_pd[FEATURES].sample(1000, random_state=42)

# pandas: melt
long_pd = sample_pd.melt(var_name="feature", value_name="value")

# polars: unpivot (o mesmo efeito)
sample_pl = df_pl.select(FEATURES).sample(n=1000, seed=42)
long_pl = sample_pl.unpivot(on=FEATURES, variable_name="feature", value_name="value")

# spark: stack (unpivot)
sample_sp = df_sp.select(*FEATURES).sample(False, 1000/df_pd.shape[0], seed=42)
expr = "stack({n}, {pairs}) as (feature, value)".format(
    n=len(FEATURES),
    pairs=", ".join([f"'{c}', {c}" for c in FEATURES])
)
long_sp = sample_sp.selectExpr(expr)

print("pandas long shape:", long_pd.shape)
print("polars long shape:", long_pl.shape)
print("spark long count:", long_sp.count())

display(long_pd.head(10))
print(long_pl.head(10))
long_sp.show(10, truncate=False)


pandas long shape: (5000, 2)
polars long shape: (5000, 2)
spark long count: 5425


,feature,value
0,price,62.538483
1,price,56.900386
2,price,42.890089
3,price,32.369843
4,price,23.606673
5,price,68.252563
6,price,39.699259
7,price,46.617663
8,price,49.024314
9,price,34.147223


shape: (10, 2)
┌─────────┬───────────┐
│ feature ┆ value     │
│ ---     ┆ ---       │
│ str     ┆ f64       │
╞═════════╪═══════════╡
│ price   ┆ 42.930468 │
│ price   ┆ 63.549227 │
│ price   ┆ 47.931766 │
│ price   ┆ 33.148437 │
│ price   ┆ 57.087296 │
│ price   ┆ 50.845615 │
│ price   ┆ 60.341    │
│ price   ┆ 75.2021   │
│ price   ┆ 38.519682 │
│ price   ┆ 59.640294 │
└─────────┴───────────┘
+-------------------+------------------+
|feature            |value             |
+-------------------+------------------+
|price              |48.10957832891635 |
|discount           |5.891365762875074 |
|promotion_intensity|4.146196902496025 |
|footfall           |191.58083448201262|
|ad_spend           |2477.471944227416 |
|price              |61.89561562807503 |
|discount           |4.124092907301649 |
|promotion_intensity|3.2353103375341856|
|footfall           |260.6342279695006 |
|ad_spend           |2429.3338399541803|
+-------------------+------------------+
only showing top 10 rows



## 8) Estatísticas descritivas

Um “raio-x” rápido das colunas numéricas.


In [9]:

# pandas
desc_pd = df_pd.describe()
display(desc_pd)

# polars
desc_pl = df_pl.describe()
print(desc_pl)

# spark
df_sp.describe().show(truncate=False)


,Unnamed: 0,price,discount,promotion_intensity,footfall,ad_spend,competitor_price,stock_level,weather_index,customer_sentiment,return_rate
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7499.500000,49.956732,6.837203,4.136523,239.445517,2504.998502,50.000493,1193.310305,7.598516,0.965801,0.063421
std,4330.271354,11.686149,3.733815,1.435314,46.325019,89.961407,12.333737,28.146760,2.032408,0.130758,0.017703
min,0.000000,20.212604,0.000000,-1.049689,70.152377,2097.020336,16.656974,1083.677929,1.889509,-0.220674,0.000000
25%,3749.750000,41.289515,3.861480,3.288099,215.277581,2454.732256,40.924934,1174.319261,6.003253,0.897901,0.051708
50%,7499.500000,49.859890,6.813337,4.420697,251.341265,2518.736486,49.996544,1193.420041,7.602752,0.977009,0.063346
75%,11249.250000,58.659467,9.713206,5.227893,272.570188,2568.142658,58.859576,1212.020553,9.202837,1.049154,0.074673
max,14999.000000,79.248053,18.757707,6.966653,348.795219,2764.819228,85.804626,1301.090400,13.451735,1.347314,0.185994


shape: (9, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆           ┆ price     ┆ discount  ┆ … ┆ stock_lev ┆ weather_i ┆ customer_ ┆ return_r │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ el        ┆ ndex      ┆ sentiment ┆ ate      │
│ str       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 15000.0   ┆ 15000.0   ┆ 15000.0   ┆ … ┆ 15000.0   ┆ 15000.0   ┆ 15000.0   ┆ 15000.0  │
│ null_coun ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ t         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ mean      ┆ 7499.5    ┆ 49.956732 ┆ 6.837203  ┆ … ┆ 1193.3103 ┆ 7.598516  


## 9) Criar nova coluna (ex.: `effective_price`)

Vamos criar `effective_price = price * (1 - discount)`  
Isso é uma proxy simples de preço efetivo após desconto.


In [10]:

# pandas
df_pd_eff = df_pd.copy()
df_pd_eff["effective_price"] = df_pd_eff["price"] * (1 - df_pd_eff["discount"])

# polars
df_pl_eff = df_pl.with_columns(
    (pl.col("price") * (1 - pl.col("discount"))).alias("effective_price")
)

# spark
df_sp_eff = df_sp.withColumn(
    "effective_price",
    F.col("price") * (F.lit(1) - F.col("discount"))
)

print("pandas effective_price head:")
display(df_pd_eff[["price","discount","effective_price"]].head(5))

print("polars effective_price head:")
print(df_pl_eff.select(["price","discount","effective_price"]).head(5))

print("spark effective_price head:")
df_sp_eff.select("price","discount","effective_price").show(5, truncate=False)


pandas effective_price head:


,price,discount,effective_price
0,45.197454,5.514259,-204.033032
1,49.327512,6.572035,-274.854643
2,47.328457,6.972713,-282.679275
3,50.964538,4.808234,-194.084877
4,44.530213,8.180216,-319.736545


polars effective_price head:
shape: (5, 3)
┌───────────┬──────────┬─────────────────┐
│ price     ┆ discount ┆ effective_price │
│ ---       ┆ ---      ┆ ---             │
│ f64       ┆ f64      ┆ f64             │
╞═══════════╪══════════╪═════════════════╡
│ 45.197454 ┆ 5.514259 ┆ -204.033032     │
│ 49.327512 ┆ 6.572035 ┆ -274.854643     │
│ 47.328457 ┆ 6.972713 ┆ -282.679275     │
│ 50.964538 ┆ 4.808234 ┆ -194.084877     │
│ 44.530213 ┆ 8.180216 ┆ -319.736545     │
└───────────┴──────────┴─────────────────┘
spark effective_price head:
+-----------------+------------------+-------------------+
|price            |discount          |effective_price    |
+-----------------+------------------+-------------------+
|45.1974540240199 |5.5142594053482386|-204.03303192572642|
|49.3275118615309 |6.57203541653068  |-274.85464310178736|
|47.32845700442001|6.9727126665304775|-282.6792746376425 |
|50.96453756162984|4.8082338496351   |-194.08487707319824|
|44.53021330035562|8.180215891669587 |-319.


## 10) Excluir colunas (drop)

Suponha que você quer remover colunas para um modelo específico.


In [11]:

cols_to_drop = ["competitor_price", "return_rate"]

# pandas
df_pd_drop = df_pd.drop(columns=cols_to_drop)

# polars
df_pl_drop = df_pl.drop(cols_to_drop)

# spark
df_sp_drop = df_sp.drop(*cols_to_drop)

print("Cols originais:", len(df_pd.columns))
print("Depois do drop | pandas:", len(df_pd_drop.columns))
print("Depois do drop | polars:", len(df_pl_drop.columns))
print("Depois do drop | spark :", len(df_sp_drop.columns))


Cols originais: 11
Depois do drop | pandas: 9
Depois do drop | polars: 9
Depois do drop | spark : 9



## 11) (Bônus) Ordenar (sort) e pegar Top N

Exemplo: Top 10 dias com maior `footfall` (tráfego).


In [12]:

# pandas
top_pd = df_pd.sort_values("footfall", ascending=False).head(10)

# polars
top_pl = df_pl.sort("footfall", descending=True).head(10)

# spark
top_sp = df_sp.orderBy(F.col("footfall").desc()).limit(10)

display(top_pd)
print(top_pl)
top_sp.show(10, truncate=False)


,Unnamed: 0,price,discount,promotion_intensity,footfall,ad_spend,competitor_price,stock_level,weather_index,customer_sentiment,return_rate
6438,6438,61.725080,9.470021,5.700920,348.795219,2598.074172,66.100890,1187.641111,8.319615,0.834384,0.052779
6425,6425,42.065761,12.831031,6.578361,335.887050,2583.344649,47.785715,1176.691450,9.041295,1.056767,0.048985
1933,1933,65.126604,10.823038,6.030652,335.651647,2551.883507,64.434440,1175.207582,8.164935,0.967707,0.038557
11581,11581,60.146520,8.670086,5.528753,334.744181,2540.728029,62.084585,1142.686251,9.302207,0.940140,0.063465
10445,10445,58.221852,9.435382,5.135991,334.473835,2528.377322,61.914346,1155.242136,7.999287,1.165335,0.035109
7210,7210,49.523798,9.086072,4.129582,333.907633,2605.706834,49.118837,1171.538911,8.384731,0.985096,0.075140
979,979,56.110589,10.043793,5.430683,333.798500,2480.739176,59.354959,1128.721699,9.791623,0.828013,0.055693
10949,10949,50.935223,8.829954,3.882293,332.727611,2545.208841,49.203001,1171.491894,8.535657,0.813855,0.074862
2212,2212,52.848691,5.084569,4.015443,330.668358,2526.230244,55.297821,1181.478856,5.385017,1.140012,0.041728
883,883,55.384071,10.073887,5.154294,330.627310,2512.930587,55.276277,1135.252395,10.119224,1.044549,0.048209


shape: (10, 11)
┌───────┬───────────┬───────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│       ┆ price     ┆ discount  ┆ promotion_ ┆ … ┆ stock_leve ┆ weather_in ┆ customer_ ┆ return_ra │
│ ---   ┆ ---       ┆ ---       ┆ intensity  ┆   ┆ l          ┆ dex        ┆ sentiment ┆ te        │
│ i64   ┆ f64       ┆ f64       ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│       ┆           ┆           ┆ f64        ┆   ┆ f64        ┆ f64        ┆ f64       ┆ f64       │
╞═══════╪═══════════╪═══════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 6438  ┆ 61.72508  ┆ 9.470021  ┆ 5.70092    ┆ … ┆ 1187.64111 ┆ 8.319615   ┆ 0.834384  ┆ 0.052779  │
│       ┆           ┆           ┆            ┆   ┆ 1          ┆            ┆           ┆           │
│ 6425  ┆ 42.065761 ┆ 12.831031 ┆ 6.578361   ┆ … ┆ 1176.69145 ┆ 9.041295   ┆ 1.056767  ┆ 0.048985  │
│ 1933  ┆ 65.126604 ┆ 10.823038 ┆ 6.030652   ┆ … ┆ 1175.20758 ┆ 8.164935   


## Fechamento

Você acabou de ver o mesmo conjunto de operações (as mais comuns de análise/engenharia de dados) em **pandas**, **polars** e **PySpark**.

Dica de cérebro:  
- pandas = “faço agora na memória”  
- polars = “faço com expressões colunar rápidas”  
- spark = “faço em cluster, com plano de execução”

